## NLP Project 13: Metaphor detection in poetry

Danila Goncharenko, 2303788

Ana Ferreira, 2308587

Mikhail Bichagov, 2304806

### This project explores the detection of metaphors in poetry using natural language processing, aiming to distinguish figurative and non-figurative language. 

We shall consider the common use of a phrase as literal use and its violation as an indicative of metaphorical use. The project initially attempts to imitate the approach of Neuman et al. (2013) published in PlusOne journal -Metaphor Identification in Large Texts Corpora- available online [`Metaphor Identification in Large Texts Corpora (plos.org)`](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0062343). So first consider the British national corpus (BNCCorpus), available through NLTK (see also [`British National Corpus, XML edition (ox.ac.uk)`](https://ota.bodleian.ox.ac.uk/repository/xmlui/handle/20.500.12024/2554)). For testing, we shall consider the annotated corpus available at https://www.eecs.uottawa.ca/~diana/resources/metaphor/type1_metaphor_annotated.txt 

In the above, the annotation at the end of the sentence i.e., @1@y   indicates whether it is a metaphor (y) or not (n). Here the presence of ‘y’ indicates that it is a metaphor, whereas “1” indicates the first head word of the sentence, which is “poise”, in the part of speech tag sequence. 


In [2]:
# Imports
import nltk
import pandas as pd
from nltk.corpus import stopwords, CategorizedPlaintextCorpusReader
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk.collocations import *

# Downloading the BNC corpus and stopwords
nltk.download('stopwords')

# Stopwords, Lemmatizer, Bi-gram
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
bigram_measures = nltk.collocations.BigramAssocMeasures()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
from nltk.corpus.reader.bnc import BNCCorpusReader

# CHANGE THE PATH
bnc_reader = BNCCorpusReader(root="C:/Users/Dan/Desktop/NLP Project/BNC/Texts", fileids=r'[A-K]/\w*/\w*\.xml')

# list_of_fileids = ['A/A0/A00.xml', 'A/A0/A01.xml']
# finder = BigramCollocationFinder.from_words(bnc_reader.words(fileids=list_of_fileids))
# scored = finder.score_ngrams(bigram_measures.raw_freq)

435261076

## Task 1

First, we shall consider the mutual information, see expression (2) in Neuman et al.’2003 paper, as a guideline to derive the metaphor-reasoning.  You can inspire from other available implementations of mutual information, in [`Collocations (nltk.org)`](https://www.nltk.org/howto/collocations.html), [`FNLP 2011: Tutorial 8: Working with corpora: mutual information (ed.ac.uk)`](http://www.inf.ed.ac.uk/teaching/courses/fnlp/lectures/8/tutorial.html). Consider the words “woman”, “use”, “dream”, “body”. Write a program that identifies all adjectives, adverbs and verbs that occur within 2 lexical units (span = 2 in the formula of mutual information) in BNC corpus and whose mutual information is equal or greater than 3, considered as the minimum statistical significance. Suggest appropriate adjustments (e.g., greater span) if no results are found to match the mutual information criterion.

In [ ]:
import nltk
from nltk.corpus.reader.bnc import BNCCorpusReader
from nltk import FreqDist, bigrams
from itertools import chain
from nltk.corpus import stopwords
from nltk.collocations import *
nltk.download('stopwords')


# Replace 'path_to_bnc_data' with the actual path to your downloaded BNC XML data
path_to_bnc_data = 'C:/Users/jklbichami/OneDrive - Valmet/Documents/School/Porgramming/NLP/bnc/Texts/news/'

# Initialize the BNC corpus reader
bnc_reader = BNCCorpusReader(root=path_to_bnc_data, fileids=r'[A-K]//w*//w*/.xml')


bigram_measures = nltk.collocations.BigramAssocMeasures()

# Here need to merge all of the words from different xlm files together.
# Its just an exmaple to show what it could look

corpus = bnc_reader.words('A1E.xml')

words_to_include = ['woman', 'use', 'dream', 'body']

# Get the words from the file in list format
es = stopwords.words('english')

g = FreqDist(bigrams(w.lower() for w in corpus if (w.isalpha() and w.lower() not in es)))

f = FreqDist()
for k in g.keys():
    if k[1] in words_to_include or k[0] in words_to_include:
        f[k] = g.get(k)

# Here we probably should not limit words by anything, except just removing stop words
u = FreqDist(w.lower() for w in corpus if (w.isalpha() and not(w.lower() in es)))

## If we have double nested list then we can use this to flatten it
chain.from_iterable(corpus)

from math import log
def mutInf(p,u1,u2,b):
    try:
        return log((float(b[p])/float(b.N()))/
                   ((float(u1[p[0]])*float(u2[p[1]]))/
                    (float(u1.N())*float(u2.N()))),
                   2)
    except:
        return

fmi = {}
for p in f.keys():
    fmi[p]  = mutInf(p,u,u,f)

fmi = {key: value for key, value in fmi.items() if value > 3}

dict(sorted(fmi.items(), key=lambda item: item[1], reverse=True))

In [16]:
# See expression (2) in Neuman et al.’2013 paper
import math
def Mutual_information(fr_B_near_A, fr_A, fr_B, sizeCorpus, span = 2):    
    '''
    Calculates Mutual information between node and collocate word
    fr_A = frequency of node word w1 (e.g., purple): 1262
    fr_B = frequency of collocate w2 (e.g., color): 115
    fr_B_near_A = frequency of collocate near the node word (e.g., color near purple): 24
    sizeCorpus = size of corpus (for instance 96,263,399)
    span = span of words (e.g., 1 to left and 1 to right of the node word: 2)
    '''

    #fr_A = 0
    #fr_B = 0
    #fr_B_near_A = 0
    #sizeCorpus = 96132981 # 96 132 981 tokens in BNC

    # Mikhail: adjusted formula to calculate
    # sizeCorpus is expected to be corpus created with bigram
    try:
        return math.log10((sizeCorpus[fr_B_near_A] * sizeCorpus.N()) / (fr_A[p[0]] * fr_B[p[1]] * span))/math.log10(2)
    except:
        return 

# Consider the words “woman”, “use”, “dream”, “body”.
consider_words = ["woman", "use", "dream", "body"]

# Identify all adjectives, adverbs and verbs that occur within 2 lexical units (span = 2 in the formula of mutual information) in BNC corpus,
# whose mutual information is equal or greater than 3, considered as the minimum statistical significance

# Suggest appropriate adjustments (e.g., greater span), if no results



## Task 2

We would like to test this process in the previous metaphor annotated dataset. For this purpose, consider the following approach. Write a program that inputs each sentence of the annotated corpus, and then reads the head word (given in the annotation), then calculate the mutual distance between the head-word and each of the first two words occurring either on the left hand side part or right hand side part of the head-word. If the average of mutual distances from head word to each of the two words situated at two lexical units is greater than 3, then we shall consider the sentence is not a metaphor, otherwise, it is a metaphor. Test this reasoning and report the result for each annotated sentence and save it in your database. Given the ground truth of the annotated dataset, calculate the corresponding accuracy, and comment on the efficiency of the proposed approach.

In [ ]:
## Mikhail: did not really understtod how to get head word. So here is idea using spacy

import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Define a sentence
## This is example with simple sentence. So we can get words from each corpus, combine them in sentence
# and then use spacy for headword search 
sentence = "The quick brown fox jumps over the lazy dog."

# Process the sentence with spaCy
doc = nlp(sentence)

# Find the root (head) of the sentence
root = None
for token in doc:
    if token.dep_ == "ROOT":
        root = token
        break

# Print the head word
if root is not None:
    print("Head word:", root.text)

## Once we got the head word, just repeat task 1 using trigram

In [19]:
url = "https://www.eecs.uottawa.ca/~diana/resources/metaphor/type1_metaphor_annotated.txt"

# Read the data into a DataFrame
df = pd.read_csv(url, delimiter='\t', header=None, names=['Text'])

# Put Annotation from Text into a separate column
df['Annotation'] = df['Text'].str.extract(r'(@\d+@y|@\d+@n)')
df['Text'] = df['Text'].str.replace(r'(@\d+@y|@\d+@n)', '', regex=True)

df

,Text,Annotation
0,poise is a club .,@1@y
1,destroying alexandria . sunlight is silence,@4@y
2,feet are no anchor . gravity sucks at the mind,@1@y
3,on the day 's horizon is a gesture of earth,@5@y
4,he said good-by as if good-by is a number .,@6@y
...,...,...
614,as the season of cold is the season of darkness,@5@n
615,"else all beasts were tigers ,",@3@y
616,without which earth is sand,@3@n
617,the sky is cloud on cloud,@2@n


In [3]:
# Input each sentence of the annotated corpus

# Read the head word (given in the annotation)

# Calculate the mutual distance between the head-word and each of the first two words
# either on the left hand side part or right hand side part of the head-word.

# If average is greater than 3, sentence is not a metaphor

# Report the result for each annotated sentence 

# Save it in your database

# Calculate the corresponding accuracy

# Comment on the efficiency of the proposed approach.


## Task 3

We consider the (adjective-noun) type of metaphor (referred to as Metaphor type III). A metaphor  assumes to occur when the categories of noun and adjective are such that one is concrete and the other one is abstract. WordStat noun categorization based on WordNet, which classifies 69,817 nouns into 25 categories, of which 13 are concrete categories (e.g., artifact) provides a database for a such categorization. It is freely available in [`Wordnet based categorization dictionary - Provalis Research`](https://provalisresearch.com/products/content-analysis-software/wordstat-dictionary/wordnet-based-categorization-dictionary/). Write a program that allows you to retrieve the category of noun and adjective / adverb in a sentence according to WordStat.

In [4]:
# adjective-noun type of metaphor is a Metaphor type III.

# If categories of noun and adjective: one is concrete and the other one is abstract,
# Then it is a Metaphor.

# WordStat noun categorization based on WordNet provides a database for a such categorization.

# Retrieve the category of noun and adjective / adverb in a sentence according to WordStat.


In [ ]:
import json
file_p = r'C:\Users\jklbichami\OneDrive - Valmet\Documents\School\Porgramming\NLP\WordNet2\WordNet Words & Phrases.CAT'

with open(file_p, 'r') as file:
    cat_data = file.readlines()

topics = [a.strip('\n') for a in cat_data if '\t' not in a ]
topics_indices = [cat_data.index(a + '\n') for a in topics]

noun_categories = [a.strip('\n')[6:] for a in cat_data if '\t\t' not in a and a.startswith('\tNOUN.')]
noun_categories_idices = [cat_data.index('\tNOUN.' + a + '\n') for a in noun_categories]

cat_df = pd.DataFrame(columns=['Type', 'Category', 'Word'])


## Mikhail: did not finish this but idea is pretty simple
## 1. We get the indeces of all topics
## 2. We get indeces for respective categories
## 3. We iterate through each topics
## 4. We iterate through their categories
## 5. Assign all results to df based on indices
## Here is examplee for Nouns
for indx, num in enumerate(noun_categories_idices):
    temp_dict = {}
    if indx == 0:
        temp_dict['Category'] = [noun_categories[indx]]*(26600-cat_data.index('ADVERBS
                                                                              \n')-2)
        ## As many times as diff between index of Verb and Noun and -2
        temp_dict['Type'] = ['Noun']*(26600-cat_data.index('ADVERBS\n')-2)
        temp_dict['Word'] = cat_data[cat_data.index('ADVERBS\n')+1:26600-1]
        cat_df = pd.concat([cat_df, pd.DataFrame(temp_dict)], ignore_index=True)
    else:
        cat_df.loc[noun_categories_idices[indx-1]:num, 'Catergory'] = noun_categories[indx]

g = FreqDist(bigrams(w.lower() for w in corpus if (w.isalpha() and w.lower() not in es)))



## Taks 4

Now we would like to imitate the procedure mentioned in Neuman’s paper for type III semaphore. Write a program that identifies the occurrence of Noun-Adjective/Adverb part-of-speech in a given sentence. Then, use WordNet lexical database to find out the number of senses of each adjective. If every adjective has one single sense, then return, no metaphor. If the Noun has no entry in wordnet, then return UNKNOWN. Otherwise (adjective has more than one sense and noun has an entry in WordNet), then identify the set S of nouns in the BNC corpus that collocate with the given Noun of the given sentence (this corresponds to a set of nouns whose mutual information value is greater or equal than 3). Next, for each element (noun) of S, use the WordStat categorization to identify those who belong to concrete class. Let S1 be a subset of S, which contains these “concrete”-category nouns. If the number of elements in S1 is large, then restrict to the first three elements who have the highest mutual information values. Finally, to find out whether, whether the sentence containing adjective A and noun N is a metaphor, we need to test the compatibility of each elements of S1 with N. If there is no elements in S1 compatible with N, then we shall consider S as a metaphor, otherwise, it is not. To evaluate this compatibility, you can use the Wu and Palmer WordNet semantic similarity already implemented in NLTK. Therefore, assume that if the Wu and Palmer semantic similarity of at least of the nouns in S1 with N is greater than a threshold 0.4, then the compatibility between S1 and N is granted. (Note this is only a very rough approximation). Write a code that implements this reasoning and test it on two simple examples of your choice. Test this process for other values of threshold values (e.g., 0.3, 0.5, 0.6) 

In [5]:
from nltk.corpus import wordnet as wn

# Imitate the procedure mentioned in Neuman’s paper for type III semaphore

# Identify the occurrence of Noun-Adjective/Adverb part-of-speech in a given sentence.

# Use WordNet lexical database to find out the number of senses of each adjective.

# If every adjective has one single sense, then no metaphor

# If the Noun has no entry in wordnet, then return UNKNOWN.

# Otherwise: adjective has more than one sense and noun has an entry in WordNet

# Identify the set S of nouns in the BNC corpus that collocate with the given Noun of the given sentence
# set of nouns: mutual information value is >= 3

# Use the WordStat categorization for each noun in S to identify those who belong to concrete class.

# Let S1 be a subset of S, which contains these “concrete”-category nouns.
# If the number of elements in S1 is large, then restrict to the first three elements who have the highest mutual information values.

# Test the compatibility of each elements of S1 with N.
# To check if the sentence containing adjective A and noun N is a metaphor

# If there is no elements in S1 compatible with N, then we shall consider S as a metaphor

# Evaluate, using the Wu and Palmer WordNet semantic similarity from NLTK.

# if semantic similarity of at least of the nouns in S1 with N >= 0.4, then the compatibility between S1 and N is granted.

# Test it on two simple examples of your choice. 

# Test this process for other values of threshold values (e.g., 0.3, 0.5, 0.6) 


## Task 5

Test the above reasoning on the first half of the dataset https://www.eecs.uottawa.ca/~diana/resources/metaphor/type1_metaphor_annotated.txt where adjective-noun type of relationship occurs. Motivate your reasoning and answers. Estimate the accuracy accordingly, and report individual results in your database.

In [7]:
# Test the above reasoning on the first half of the dataset

# Motivate your reasoning and answers.

# Estimate the accuracy accordingly, and report individual results in your database.


## Task 6

Instead of the calculus of the semantic similarity between N and each elements of S1 in step 4, we would like to use the wordnet domain of each individual words. For this purpose, download the wordnet domain from [`WordNet Domains (fbk.eu)`](https://wndomains.fbk.eu/download.html). Therefore, the compatibility between N and an element N1 of S1 is granted if N and N1 belong to the same wordnet domain. Write a program that allows you to implement this reasoning and test it on simple sentences of your choice.

In [9]:
# Use the wordnet domain of each individual words, instead of similarity between N and each elements of S1

# Download the wordnet domain

# If N and N1 belong to the same wordnet domain, compatibility between N and an element N1 of S1 is granted

# Test it on simple sentences of your choice


## Task 7

Test the reasoning of 6) on the same subset of annotated metaphor dataset used in 5) and compare the performance in terms of accuracy. Save individual results in your database as well.

In [10]:
# Test the reasoning of 6) on dataset used in 5)

# Compare the performance in terms of accuracy

# Save individual results in your database as well


## Task 8

Repeat 6) and 7) when using Reuter corpus (also accessible via NLTK) instead of BNC corpus. Conclude on the impact of the corpus on the accuracy of metaphor identification.  

In [11]:
# Repeat 6) and 7) when using Reuter corpus from NLTK

# Conclude on the impact of the corpus on the accuracy of metaphor identification


## Task 9

You may want to enhance the reasoning of any of project specifications above, feel free to suggest any state-of-the-art approach that you judge relevant and accommodate to achieve the goal accordingly. Motivate your choice by concise literature review. Use appropriate literature to discuss your findings.

In [14]:
# Enhance the reasoning of any of project specifications above

# Suggest any relevant state-of-the-art approach

# Motivate your choice by concise literature review. 

# Use appropriate literature to discuss your findings.
